# Circuit Trace Demo
Use `rg-tracer trace` to materialise circuit traces for a run.

## Attribution Delta Panel
Use attribution metrics to contextualise circuit traces across training phases.

In [ ]:
NO_ATTR_RUNS_MSG = 'No runs with attribution metrics found; execute a self-play run first.'import jsonfrom pathlib import Pathrun_dir = Path('runs').resolve()if not run_dir.exists() or not run_dir.is_dir():    raise RuntimeError(NO_ATTR_RUNS_MSG)existing = sorted(    path    for path in run_dir.iterdir()    if path.is_dir() and (path / 'attr_metrics.jsonl').exists())if not existing:    raise RuntimeError(NO_ATTR_RUNS_MSG)latest = existing[-1]metrics_path = latest / 'attr_metrics.jsonl'records = [    json.loads(line)    for line in metrics_path.read_text(encoding='utf8').splitlines()    if line.strip()]top_3 = sorted(records, key=lambda item: item['delta_alignment'], reverse=True)[:3]top_3